In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)

In [2]:
## get the variables

PATH_1=os.path.join('..','..','..','Weather_Data','ERA5','2013','era_2013.nc')
humidity_data = xr.open_dataset(PATH_1) ## variables
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2013-01-01 ... 2013-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
    expver          (valid_time) <U4 47kB ...
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:41 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
humidity_data= humidity_data.drop_vars('number') ## not sure what these two are
humidity_data= humidity_data.drop_vars('expver')
humidity_data

<xarray.Dataset> Size: 542MB
Dimensions:         (valid_time: 2920, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 23kB 2013-01-01 ... 2013-12-3...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 271MB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:41 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
humidity_data.to_dataframe()

r         q
valid_time          pressure_level latitude longitude                     
2013-01-01 00:00:00 1000.0         49.25    -124.85    88.067215  0.003714
                                            -124.60    88.407059  0.003647
                                            -124.35    86.864090  0.003577
                                            -124.10    84.110184  0.003525
                                            -123.85    82.459793  0.003551
...                                                          ...       ...
2013-12-31 21:00:00 1000.0         24.50    -68.10     78.253906  0.014256
                                            -67.85     78.177734  0.014252
                                            -67.60     78.871094  0.014353
                                            -67.35     79.355469  0.014388
                                            -67.10     79.875000  0.014456

[67744000 rows x 2 columns]

In [5]:
## resample the data to 1 year average

year_avg = humidity_data.resample(valid_time='1YE').mean(dim='valid_time')
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, pressure_level: 1, latitude: 100,
                     longitude: 232)
Coordinates:
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2013-12-31
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
    q               (valid_time, pressure_level, latitude, longitude) float32 93kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:41 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
year_avg.to_dataframe()

r         q
valid_time pressure_level latitude longitude                     
2013-12-31 1000.0         49.25    -124.85    83.565842  0.006347
                                   -124.60    80.759293  0.006380
                                   -124.35    78.100601  0.006257
                                   -124.10    75.812256  0.006139
                                   -123.85    73.366051  0.006156
...                                                 ...       ...
                          24.50    -68.10     77.566742  0.014247
                                   -67.85     77.494415  0.014222
                                   -67.60     77.489494  0.014202
                                   -67.35     77.497002  0.014182
                                   -67.10     77.510765  0.014163

[23200 rows x 2 columns]

In [7]:
year_avg = year_avg.squeeze(dim="pressure_level") ## drop the pressure level dimension
year_avg

<xarray.Dataset> Size: 188kB
Dimensions:         (valid_time: 1, latitude: 100, longitude: 232)
Coordinates:
    pressure_level  float64 8B 1e+03
  * latitude        (latitude) float64 800B 49.25 49.0 48.75 ... 25.0 24.75 24.5
  * longitude       (longitude) float64 2kB -124.8 -124.6 ... -67.35 -67.1
  * valid_time      (valid_time) datetime64[ns] 8B 2013-12-31
Data variables:
    r               (valid_time, latitude, longitude) float32 93kB 83.57 ... ...
    q               (valid_time, latitude, longitude) float32 93kB 0.006347 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-05T20:41 GRIB to CDM+CF via cfgrib-0.9.1...

In [8]:
SHAPE_PATH=os.path.join('..','..','..','Shapefiles','county_shapefiles','2013_county_shapefile','cb_2013_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,001,00161526,0500000US01001,01001,Autauga,06,1539584443,25773561,"POLYGON ((-86.92120 32.65754, -86.92035 32.658..."
1,01,009,00161530,0500000US01009,01009,Blount,06,1670041814,15077461,"POLYGON ((-86.96336 33.85822, -86.95967 33.857..."
2,01,023,00161537,0500000US01023,01023,Choctaw,06,2365954803,19059247,"POLYGON ((-88.47323 31.89386, -88.46888 31.930..."
3,01,033,00161542,0500000US01033,01033,Colbert,06,1534878355,80029923,"POLYGON ((-88.13925 34.58779, -88.13872 34.589..."
4,01,049,00161550,0500000US01049,01049,DeKalb,06,2012664422,4121499,"POLYGON ((-86.11889 34.40384, -86.11002 34.403..."
...,...,...,...,...,...,...,...,...,...,...
3229,13,171,00326713,0500000US13171,13171,Lamar,06,475262687,6046028,"POLYGON ((-84.26917 33.03144, -84.25906 33.031..."
3230,18,007,00450403,0500000US18007,18007,Benton,06,1052616465,236033,"POLYGON ((-87.52650 40.47716, -87.52638 40.491..."
3231,16,053,00395662,0500000US16053,16053,Jerome,06,1546716831,12848211,"POLYGON ((-114.61740 42.85081, -114.59511 42.8..."
3232,17,095,00424249,0500000US17095,17095,Knox,06,1855506498,8834347,"POLYGON ((-90.44412 40.72857, -90.44400 40.729..."


In [9]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-86.92120 32.65754, -86.92035 32.658...
1       POLYGON ((-86.96336 33.85822, -86.95967 33.857...
2       POLYGON ((-88.47323 31.89386, -88.46888 31.930...
3       POLYGON ((-88.13925 34.58779, -88.13872 34.589...
4       POLYGON ((-86.11889 34.40384, -86.11002 34.403...
                              ...                        
3229    POLYGON ((-84.26917 33.03144, -84.25906 33.031...
3230    POLYGON ((-87.52650 40.47716, -87.52638 40.491...
3231    POLYGON ((-114.61740 42.85081, -114.59511 42.8...
3232    POLYGON ((-90.44412 40.72857, -90.44400 40.729...
3233    POLYGON ((-85.65975 41.75910, -85.65074 41.759...
Name: geometry, Length: 3234, dtype: geometry

In [10]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0  -86.921196  32.657542
     1  -86.920352  32.658563
     2  -86.920411  32.660077
     3  -86.917595  32.664169
     4  -86.914610  32.664355
...            ...        ...
3233 66 -85.658036  41.675677
     67 -85.657951  41.676459
     68 -85.658614  41.696900
     69 -85.658770  41.704005
     70 -85.659750  41.759101

[893880 rows x 2 columns]

In [11]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            county_df: Dataframe consisting the values of the variables interpolated in the county.
    ''' 
    longitude= lat_lon.loc[(var), 'x']  # get the longitude 
    extract_val=len(longitude)        ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    
    latitude= lat_lon.loc[(var), 'y']   # get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)
    year_avg_finer
    
    df = year_avg_finer.squeeze(dim="valid_time") ## remove the valid_time dimension, note that this will appear as a column
                                                  ## after removed

    ## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=df.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)

    initial_df=pd.concat(get_vals)
    initial_df=initial_df.drop('valid_time', axis=1) ## remove the valid_time column 
    
    mean=initial_df.mean() ## find the average of the 100 (or less) values
    average_df = pd.DataFrame(mean).T 

    county_df= average_df.convert_dtypes() ## the dtype of the mean values will be object, so convert that to float
    county_df
    
    return county_df

In [12]:
%%time

## include all counties

df_list=[]

for i in range(0,3234): ## A total of 3233 counties. index in shape file starts from 0 and ends in 3232
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 5min 50s
Wall time: 5min 56s


In [13]:
concatenate_df=pd.concat(df_list)
final_df=concatenate_df.reset_index(drop=True)
final_df = final_df.drop('pressure_level', axis=1)
final_df

,r,q
0,68.557795,0.009679
1,71.747599,0.00928
2,67.71892,0.009755
3,69.924852,0.008886
4,73.391248,0.00895
...,...,...
3229,72.0608,0.009594
3230,68.599697,0.006859
3231,48.074144,0.003829
3232,67.376517,0.006681


In [14]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

STATEFP       object
COUNTYFP      object
COUNTYNS      object
AFFGEOID      object
GEOID         object
NAME          object
LSAD          object
ALAND          int64
AWATER         int64
geometry    geometry
fips           int32
dtype: object

In [15]:
county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Autauga,"POLYGON ((-86.92120 32.65754, -86.92035 32.658...",1001
1,Blount,"POLYGON ((-86.96336 33.85822, -86.95967 33.857...",1009
2,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023
3,Colbert,"POLYGON ((-88.13925 34.58779, -88.13872 34.589...",1033
4,DeKalb,"POLYGON ((-86.11889 34.40384, -86.11002 34.403...",1049
...,...,...,...
3229,Lamar,"POLYGON ((-84.26917 33.03144, -84.25906 33.031...",13171
3230,Benton,"POLYGON ((-87.52650 40.47716, -87.52638 40.491...",18007
3231,Jerome,"POLYGON ((-114.61740 42.85081, -114.59511 42.8...",16053
3232,Knox,"POLYGON ((-90.44412 40.72857, -90.44400 40.729...",17095


In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,r,q
0,Autauga,"POLYGON ((-86.92120 32.65754, -86.92035 32.658...",1001,68.557795,0.009679
1,Blount,"POLYGON ((-86.96336 33.85822, -86.95967 33.857...",1009,71.747599,0.00928
2,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,67.71892,0.009755
3,Colbert,"POLYGON ((-88.13925 34.58779, -88.13872 34.589...",1033,69.924852,0.008886
4,DeKalb,"POLYGON ((-86.11889 34.40384, -86.11002 34.403...",1049,73.391248,0.00895
...,...,...,...,...,...
3229,Lamar,"POLYGON ((-84.26917 33.03144, -84.25906 33.031...",13171,72.0608,0.009594
3230,Benton,"POLYGON ((-87.52650 40.47716, -87.52638 40.491...",18007,68.599697,0.006859
3231,Jerome,"POLYGON ((-114.61740 42.85081, -114.59511 42.8...",16053,48.074144,0.003829
3232,Knox,"POLYGON ((-90.44412 40.72857, -90.44400 40.729...",17095,67.376517,0.006681


In [17]:
county_df=merged_df.dropna()
county_df

,NAME,geometry,fips,r,q
0,Autauga,"POLYGON ((-86.92120 32.65754, -86.92035 32.658...",1001,68.557795,0.009679
1,Blount,"POLYGON ((-86.96336 33.85822, -86.95967 33.857...",1009,71.747599,0.00928
2,Choctaw,"POLYGON ((-88.47323 31.89386, -88.46888 31.930...",1023,67.71892,0.009755
3,Colbert,"POLYGON ((-88.13925 34.58779, -88.13872 34.589...",1033,69.924852,0.008886
4,DeKalb,"POLYGON ((-86.11889 34.40384, -86.11002 34.403...",1049,73.391248,0.00895
...,...,...,...,...,...
3229,Lamar,"POLYGON ((-84.26917 33.03144, -84.25906 33.031...",13171,72.0608,0.009594
3230,Benton,"POLYGON ((-87.52650 40.47716, -87.52638 40.491...",18007,68.599697,0.006859
3231,Jerome,"POLYGON ((-114.61740 42.85081, -114.59511 42.8...",16053,48.074144,0.003829
3232,Knox,"POLYGON ((-90.44412 40.72857, -90.44400 40.729...",17095,67.376517,0.006681


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,r,q
0,Autauga,1001,68.557794915745092,0.009679442452573
1,Blount,1009,71.747598980783337,0.009280396826593
2,Choctaw,1023,67.718920244947100,0.009754857672470
3,Colbert,1033,69.924852094900331,0.008885893143712
4,DeKalb,1049,73.391248374822894,0.008950312754404
...,...,...,...,...
3229,Lamar,13171,72.060800340734659,0.009593548909699
3230,Benton,18007,68.599696838466372,0.006859466908380
3231,Jerome,16053,48.074144353630238,0.003829197280865
3232,Knox,17095,67.376516705825679,0.006681191536921


In [20]:
county_var = county_var.sort_values(by='fips')
county_var

,NAME,fips,r,q
0,Autauga,1001,68.557794915745092,0.009679442452573
769,Baldwin,1003,73.803152431789783,0.011471035157888
978,Barbour,1005,72.145635501814283,0.010315539280437
2485,Bibb,1007,70.221184034130218,0.009593326001242
1,Blount,1009,71.747598980783337,0.009280396826593
...,...,...,...,...
1717,Sweetwater,56037,55.906575079687727,0.003651022848041
1341,Teton,56039,67.843922444923038,0.003682541936500
1718,Uinta,56041,57.267945562216084,0.003626885007184
1719,Washakie,56043,53.364110316466430,0.003940944770736


In [21]:
county_var.to_pickle('era_2013.pkl')